In [125]:
# Example usage:
# m, f = convert_simple_to_problem_format("path/to/simple_format_file.txt")
# selected_facilities, total_cost = solve_pyomo(m, f)
# m, f = convert_orlib_cap_to_problem_format("/Users/krupke/Repositories/HardProblemBench/tasks/facility_location/CLSC/3133GapCS.txt")
# m,f

In [4]:
from algbench import Benchmark, read_as_pandas

benchmark = Benchmark("./.data", hide_output=False)

In [17]:
t = read_as_pandas(
    "./.data",
    lambda x: {
        "instance_name": x["parameters"]["args"]["instance_name"],
        "model": x["parameters"]["args"]["model"],
        "time": x["runtime"],
        "objective": x["result"]["objective"],
        "bound": x["result"]["bound"],
    },
)
t

,instance_name,model,time,objective,bound
0,KoerkelGhosh-asym/500/b/ga500b-4,solve1,33.249497,559202,531859
1,KoerkelGhosh-asym/500/b/ga500b-4,solve2,32.421462,559103,531882
2,KoerkelGhosh-asym/500/b/ga500b-4,solve3,32.132297,562834,531884
3,KoerkelGhosh-asym/500/b/ga500b-5,solve1,33.250591,560911,532094
4,KoerkelGhosh-asym/500/b/ga500b-5,solve2,32.188633,564773,531917
...,...,...,...,...,...
130,KoerkelGhosh-asym/750/b/ga750b-3,solve2,35.112618,1138903,786955
131,KoerkelGhosh-asym/750/b/ga750b-3,solve3,35.116273,1043070,786955
132,KoerkelGhosh-asym/750/c/ga750c-1,solve1,36.721125,1248509,796492
133,KoerkelGhosh-asym/750/c/ga750c-1,solve2,34.899835,2585201,793059


In [2]:
benchmark.front()

{'result': {'objective': 23468, 'bound': 23468},
 'timestamp': '2025-06-04T18:51:00.089656',
 'runtime': 0.3599112033843994,
 'stdout': [],
 'stderr': [],
 'logging': [],
 'env_fingerprint': '5de30a459f30cb188ccface596923cf8eb699f7c',
 'args_fingerprint': '8bdd02ca05185e25bc033da70941bd5374b3e696',
 'parameters': {'func': 'solve',
  'args': {'instance_name': 'BildeKrarup/BildeKrarup/B/B1',
   'model': 'solve1',
   'time_limit': 30,
   'instance_': "instance_uid='BildeKrarup/BildeKrarup/B/B1' origin='https://resources.mpi-inf.mpg.de/departments/d1/projects/benchmarks/UflLib/data/bench/BildeKrarup.tgz' comment='' num_clients=100 num_facilities=50 is_integral=True opening_cost=[4751, 3011, 7433, 3947, 8799, 9776, 8951, 9182, 7895, 9103, 1736, 3485, 9627, 2090, 2501, 3527, 5234, 8975, 3751, 6554, 9543, 7599, 6022, 4293, 6957, 1866, 7764, 8018, 4583, 3907, 7729, 1244, 2079, 6172, 7198, 2024, 7351, 6174, 3889, 7315, 6641, 7092, 9907, 1373, 9155, 2961, 4548, 7670, 2782, 6664] allocation_cost=

In [3]:
from instance_schema import FacilityLocationInstance, iter_all_instances
from pathlib import Path

In [ ]:
for instance in iter_all_instances(
    Path(
        "/home/krupke/Repositories/instance_repository_backend/REPOSITORY/flp/instances/KoerkelGhosh-asym"
    )
):
    print(instance.instance_uid)
    solve1(instance)

KoerkelGhosh-asym/500/a/ga500a-5

Starting CP-SAT solver v9.12.4544
Parameters: log_search_progress: true
Setting number of workers to 24

Initial optimization model '': (model_fingerprint: 0xdfda128222036912)
#Variables: 250'500 (#bools: 250'500 in objective)
  - 250'500 Booleans in [0,1]
#kLinear2: 250'000
#kLinearN: 500 (#terms: 250'000)

Starting presolve at 0.05s
  2.75e-01s  0.00e+00d  [DetectDominanceRelations] 
  8.77e-01s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  2.01e-03s  0.00e+00d  [ExtractEncodingFromLinear] #potential_supersets=500 
  7.71e-03s  0.00e+00d  [DetectDuplicateColumns] 
  1.37e-01s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 752'000 nodes and 1'251'000 arcs.
[Symmetry] Symmetry computation done. time: 0.244097 dtime: 0.153994
[SAT presolve] num removable Booleans: 0 / 250500
[SAT presolve] num trivial clauses: 0
[SAT presolve] [0s] clauses:250000 literals:500000 vars:250500 one_side_vars:250500 s

In [4]:
def solve2(instance: FacilityLocationInstance) -> tuple[list[int], int]:
    """
    Solve the facility location problem with a greedy heuristic.
    :param m: serving cost matrix
    :param f: opening cost vector
    :param k: number of facilities to open
    :return: tuple of selected facilities and total cost
    """
    from ortools.sat.python import cp_model

    model = cp_model.CpModel()
    # x[j] = 1 if facility j is open
    x = [model.NewBoolVar(f"x[{j}]") for j in instance.iter_facilities()]
    # y[i][j] = 1 if client i is served by facility j
    y = [
        [model.NewBoolVar(f"y[{i}][{j}]") for j in instance.iter_facilities()]
        for i in instance.iter_clients()
    ]
    # Each client must be served by exactly one facility
    for i in instance.iter_clients():
        model.Add(sum(y[i][j] for j in instance.iter_facilities()) == 1)
    # Each facility can serve clients only if it is open
    for j in instance.iter_facilities():
        model.Add(
            sum(y[i][j] for i in instance.iter_clients()) <= x[j] * instance.num_clients
        )

    # Add objective function
    model.Minimize(
        sum(instance.opening_cost[j] * x[j] for j in instance.iter_facilities())
        + sum(
            instance.get_allocation_cost(client=i, facility=j) * y[i][j]
            for i in instance.iter_clients()
            for j in instance.iter_facilities()
        )
    )
    # Solve the model
    solver = cp_model.CpSolver()
    solver.parameters.log_search_progress = True
    solver.Solve(model)
    # Extract the solution
    selected_facilities = [
        j for j in instance.iter_facilities() if solver.Value(x[j]) == 1
    ]
    total_cost = solver.ObjectiveValue()
    return selected_facilities, int(total_cost)


for instance in iter_all_instances(
    Path(
        "/home/krupke/Repositories/instance_repository_backend/REPOSITORY/flp/instances/BildeKrarup/BildeKrarup"
    )
):
    print(instance.instance_uid)
    solve2(instance)

BildeKrarup/BildeKrarup/B/B1

Starting CP-SAT solver v9.12.4544
Parameters: log_search_progress: true
Setting number of workers to 24

Initial optimization model '': (model_fingerprint: 0xcd4868203f697c26)
#Variables: 5'050 (#bools: 5'044 in objective)
  - 5'050 Booleans in [0,1]
#kLinearN: 150 (#terms: 10'050)

Starting presolve at 0.00s
  1.39e-03s  0.00e+00d  [DetectDominanceRelations] 
  4.00e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  1.14e-05s  0.00e+00d  [ExtractEncodingFromLinear] #potential_supersets=100 
  9.47e-05s  0.00e+00d  [DetectDuplicateColumns] 
  5.53e-04s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 15'250 nodes and 25'100 arcs.
[Symmetry] Symmetry computation done. time: 0.000923851 dtime: 0.00248982
[SAT presolve] num removable Booleans: 0 / 5050
[SAT presolve] num trivial clauses: 0
[SAT presolve] [0s] clauses:5000 literals:10000 vars:5050 one_side_vars:5050 simple_definition:0 singleton_clauses:0

In [5]:
def solve3(instance: FacilityLocationInstance) -> tuple[list[int], int]:
    """
    Solve the facility location problem with a greedy heuristic.
    :param m: serving cost matrix
    :param f: opening cost vector
    :param k: number of facilities to open
    :return: tuple of selected facilities and total cost
    """
    from ortools.sat.python import cp_model

    model = cp_model.CpModel()
    # x[j] = 1 if facility j is open
    x = [model.NewBoolVar(f"x[{j}]") for j in instance.iter_facilities()]
    # y[i][j] = 1 if client i is served by facility j
    y = [
        [model.NewBoolVar(f"y[{i}][{j}]") for j in instance.iter_facilities()]
        for i in instance.iter_clients()
    ]
    # Each client must be served by exactly one facility
    for i in instance.iter_clients():
        model.Add(sum(y[i][j] for j in instance.iter_facilities()) == 1)
    # Each facility can serve clients only if it is open
    for j in instance.iter_facilities():
        model.Add(sum(y[i][j] for i in instance.iter_clients()) == 0).OnlyEnforceIf(
            ~x[j]
        )

    # Add objective function
    model.Minimize(
        sum(instance.opening_cost[j] * x[j] for j in instance.iter_facilities())
        + sum(
            instance.get_allocation_cost(client=i, facility=j) * y[i][j]
            for i in instance.iter_clients()
            for j in instance.iter_facilities()
        )
    )
    # Solve the model
    solver = cp_model.CpSolver()
    solver.parameters.log_search_progress = True
    solver.Solve(model)
    # Extract the solution
    selected_facilities = [
        j for j in instance.iter_facilities() if solver.Value(x[j]) == 1
    ]
    total_cost = solver.ObjectiveValue()
    return selected_facilities, int(total_cost)


for instance in iter_all_instances(
    Path(
        "/home/krupke/Repositories/instance_repository_backend/REPOSITORY/flp/instances/BildeKrarup/BildeKrarup"
    )
):
    print(instance.instance_uid)
    solve3(instance)

BildeKrarup/BildeKrarup/B/B1

Starting CP-SAT solver v9.12.4544
Parameters: log_search_progress: true
Setting number of workers to 24

Initial optimization model '': (model_fingerprint: 0xdab0f06e908622d5)
#Variables: 5'050 (#bools: 5'044 in objective)
  - 5'050 Booleans in [0,1]
#kLinearN: 150 (#enforced: 50) (#terms: 10'000)

Starting presolve at 0.00s
  1.31e-03s  0.00e+00d  [DetectDominanceRelations] 
  3.53e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  8.94e-06s  0.00e+00d  [ExtractEncodingFromLinear] #potential_supersets=100 
  6.59e-05s  0.00e+00d  [DetectDuplicateColumns] 
  5.51e-04s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 15'250 nodes and 25'100 arcs.
[Symmetry] Symmetry computation done. time: 0.000861667 dtime: 0.00248982
[SAT presolve] num removable Booleans: 0 / 5050
[SAT presolve] num trivial clauses: 0
[SAT presolve] [0s] clauses:5000 literals:10000 vars:5050 one_side_vars:5050 simple_definition:0 sin

In [129]:
def solve_pyomo(m: list[list[int]], f: list[int]) -> tuple[list[int], int]:
    """
    Solve the facility location problem using Pyomo with HiGHS solver.
    :param m: serving cost matrix
    :param f: opening cost vector
    :return: tuple of selected facilities and total cost
    """
    import pyomo.environ as pyo

    # Create model
    model = pyo.ConcreteModel()

    # Define index sets
    model.facilities = pyo.RangeSet(
        0, len(f) - 1
    )  # facilities indexed from 0 to len(f)-1
    model.clients = pyo.RangeSet(0, len(m) - 1)  # clients indexed from 0 to len(m)-1

    # Decision variables
    # x[j] = 1 if facility j is open
    model.x = pyo.Var(model.facilities, domain=pyo.Binary)
    # y[i,j] = 1 if client i is served by facility j
    model.y = pyo.Var(model.clients, model.facilities, domain=pyo.Binary)

    # Each client must be served by exactly one facility
    def client_assignment_rule(model, i):
        return sum(model.y[i, j] for j in model.facilities) == 1

    model.client_assignment = pyo.Constraint(model.clients, rule=client_assignment_rule)

    # Each facility can serve clients only if it is open
    def facility_open_rule(model, i, j):
        return model.y[i, j] <= model.x[j]

    model.facility_open = pyo.Constraint(
        model.clients, model.facilities, rule=facility_open_rule
    )

    # Objective function
    def objective_rule(model):
        facility_cost = sum(f[j] * model.x[j] for j in model.facilities)
        service_cost = sum(
            m[i][j] * model.y[i, j] for i in model.clients for j in model.facilities
        )
        return facility_cost + service_cost

    model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

    # Solve the model
    solver = pyo.SolverFactory("appsi_highs")
    results = solver.solve(model, tee=True)  # tee=True to display solver output

    # Check solution status
    if (
        results.solver.status == pyo.SolverStatus.ok
        and results.solver.termination_condition == pyo.TerminationCondition.optimal
    ):
        # Extract the solution
        selected_facilities = [
            j for j in model.facilities if pyo.value(model.x[j]) > 0.5
        ]
        total_cost = pyo.value(model.objective)

        return selected_facilities, int(total_cost)
    else:
        print(f"Solver status: {results.solver.status}")
        print(f"Termination condition: {results.solver.termination_condition}")
        return [], 0


# Example usage:
solve_pyomo(m, f)

Running HiGHS 1.10.0 (git hash: fd86653): Copyright (c) 2025 HiGHS under MIT licence terms
RUN!
MIP  has 12000 rows; 11523 cols; 34500 nonzeros; 11523 integer variables (11523 binary)
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [1e+00, 1e+03]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+00]
Presolving model
12000 rows, 11523 cols, 34500 nonzeros  0s
12000 rows, 11523 cols, 34500 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   12000 rows
   11523 cols (11523 binary, 0 integer, 0 implied int., 0 continuous)
   34500 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src 

([0, 2, 3, 5, 9, 11, 13, 14, 15, 16, 18, 20, 21, 22], 18865)

In [130]:
def solve2_pyomo(m: list[list[int]], f: list[int]) -> tuple[list[int], int]:
    """
    Solve the facility location problem using Pyomo with HiGHS solver.
    This implements the same model as solve2 with the constraint that
    the sum of clients served by a facility is bounded by the number of clients
    multiplied by whether the facility is open.

    :param m: serving cost matrix
    :param f: opening cost vector
    :return: tuple of selected facilities and total cost
    """
    import pyomo.environ as pyo

    # Create model
    model = pyo.ConcreteModel()

    # Define index sets
    model.facilities = pyo.RangeSet(
        0, len(f) - 1
    )  # facilities indexed from 0 to len(f)-1
    model.clients = pyo.RangeSet(0, len(m) - 1)  # clients indexed from 0 to len(m)-1

    # Decision variables
    # x[j] = 1 if facility j is open
    model.x = pyo.Var(model.facilities, domain=pyo.Binary)
    # y[i,j] = 1 if client i is served by facility j
    model.y = pyo.Var(model.clients, model.facilities, domain=pyo.Binary)

    # Each client must be served by exactly one facility
    def client_assignment_rule(model, i):
        return sum(model.y[i, j] for j in model.facilities) == 1

    model.client_assignment = pyo.Constraint(model.clients, rule=client_assignment_rule)

    # Each facility can serve clients only if it is open (using aggregate constraint)
    # This matches the approach in solve2 that uses: sum(y[i][j]) <= x[j] * len(m)
    def facility_capacity_rule(model, j):
        return sum(model.y[i, j] for i in model.clients) <= model.x[j] * len(m)

    model.facility_capacity = pyo.Constraint(
        model.facilities, rule=facility_capacity_rule
    )

    # Objective function
    def objective_rule(model):
        facility_cost = sum(f[j] * model.x[j] for j in model.facilities)
        service_cost = sum(
            m[i][j] * model.y[i, j] for i in model.clients for j in model.facilities
        )
        return facility_cost + service_cost

    model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

    # Solve the model
    solver = pyo.SolverFactory("appsi_highs")
    results = solver.solve(model, tee=True)  # tee=True to display solver output

    # Check solution status
    if (
        results.solver.status == pyo.SolverStatus.ok
        and results.solver.termination_condition == pyo.TerminationCondition.optimal
    ):
        # Extract the solution
        selected_facilities = [
            j for j in model.facilities if pyo.value(model.x[j]) > 0.5
        ]
        total_cost = pyo.value(model.objective)

        return selected_facilities, int(total_cost)
    else:
        print(f"Solver status: {results.solver.status}")
        print(f"Termination condition: {results.solver.termination_condition}")
        return [], 0


# Example usage:
solve2_pyomo(m, f)

Running HiGHS 1.10.0 (git hash: fd86653): Copyright (c) 2025 HiGHS under MIT licence terms
RUN!
MIP  has 523 rows; 11523 cols; 23023 nonzeros; 11523 integer variables (11523 binary)
Coefficient ranges:
  Matrix [1e+00, 5e+02]
  Cost   [1e+00, 1e+03]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+00]
Presolving model
523 rows, 11523 cols, 23023 nonzeros  0s
523 rows, 11523 cols, 23023 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   523 rows
   11523 cols (11523 binary, 0 integer, 0 implied int., 0 continuous)
   23023 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. I

([0, 2, 3, 5, 9, 11, 13, 14, 15, 16, 18, 20, 21, 22], 18865)